# <span style="color:#336699">Introduction to Data Science - CAP 394 [<img src="./img/icone_cap394.svg" alt="CAP394 - Introduction to Data Science" style="height:120px;" align="right">](http://www.lac.inpe.br/~rafael.santos/cap394.html)</span>


<hr style="border:2px solid #0077b9;">

   
<h1> <p align="right"> Exploratory Analysis of Meteorological Radar Dataset </p> </h1>
   
<p align='center'> Project for course  Introduction to Data Science - CAP 394</p>

#### Instructors: 
* Dr. Gilberto Queiroz 
* Dr. Rafael Santos

#### Student: 
* Helvecio Leal Neto



## Schedule
     - [x] Data reading
     - [x] Data visualization
     - [x] Preprocessing
     - [x] Cluster detection algorithm
     - [x] Relational Directions
     - [x] Create Data Functions
     - [x] Visualization Radar image
     - [x] Statistics and Results
     - [ ] Final analysis and project elaboration

### Index

  [About](#about) <br></br>
  [The Data](#the_data)<br></br>
  [Reading Data](#reading_data)<br></br>
  
 <hr style="border:1px solid #0077b9;">

<a id='about'></a>
## About

<p align="justify"> <br>
This work is part of the suite of applications for the Introduction to Data Science (CAP-394) course offered by the National Institute for Space Research.

The purpose of this paper is to perform an exploratory analysis of radar data, gathering raw data, processing it and understanding the dynamics of cloud location from its center of mass, as well as counting the number of clusters per period.
At the end of the exploratory process it will be possible to estimate the behavior of the clouds and relate their geographical location as well as correlate average precipitation rates per hour.

<hr style="border:1px solid #0077b9;">

<a id='the_data'></a>
## The Data

<p align="justify"><br>
The data analyzed in this example consists of NC (NetCDF4) files. Each file corresponds to Rain Rate measurements collected by the Amazon Protection System (SIPAM) radar during the GoAmazon experiment periods, starting from January 2014 to December 2015, such files contain data on 12-minute time intervals. The dataset provides rain data over 2.5km from S-Band Radar located in lat: -3.148556, lon: -59.992000. Data dimensionality consists of a two-dimensional (241x241) matrix containing rain rate values, the Radar Radius range is approximately 240 km, covering an area of 1,500 km² over the state of Amazonas (Brazil).
<p align="justify"><br>
Overview:

"S-band radar volumes from Manaus, Brazil were shared by SIPAM for the time period beginning in January 2014 to December 2015 partially coinciding with the GOAmazon field campaign. The corrected radar reflectivity from each volume were interpolated to a fixed grid and the rainfall products were generated. A single Z-R relation (Z=174.8R^1.56) was created using 2014 wet-season impact disdrometer data and applied to the 2.5 km SIPAM Manaus S-Band CAPPI data to generate rain rates for each radar volume. This is version 2.0a of the dataset, which benefits from improved quality control procedures to remove non-meteorological data. " Schumacher</p>


https://www.arm.gov/research/campaigns/amf2014goamazon <br>

* Courtney Schumacher	S-band Radar - Rain Rates

<img src="img/SIPAM.png" align="center" alt="Drawing" style="width: 600px;">


[Figure 01: REDEMET - Radar Manaus](https://www.redemet.aer.mil.br/?i=produtos&p=radares-meteorologicos)
<hr style="border:1px solid #0077b9;">

## Questions about the data

1. How can I do the data acquisition?
2. How can I open the data?
3. What are the functions to view the data content?
4. How to format data efficiently?
5. Which precipitation rate intensity thresholds are interesting?
6. What techniques for clustering can be applied to the dataset?
7. How to convert values in Rain Rate Fall to DBz?
8. What geographic applications can be associated with data?
9. How to organize and process data efficiently?
10. How to use a library to plot radar images?
11. How many clusters per hour and month are in the dataset?
12. What is the accumulated rain fall present in the data?
13. What are the directional relationships of each cluster?

<hr style="border:1px solid #0077b9;">

<a id='reading_data'></a>
## 1. Basic read process

### 1.1 Reading the metadata file

In [ ]:
import os
import xarray as xr

In [ ]:
### open files day 2014 / 01 / 03
day = 20150101

### path contains dataset
path = 'data/radar/'

#read first file to extract static variable values
first_file = path+str(day)+'/'+str(os.listdir(path+str(day))[0])
    
try:
    xds = xr.open_dataset(first_file)   
except Exception as e:
    print('File not found')
xds

In [ ]:
# settingup the variables
rr = xds.rain_rate                  ### Matrix with preciptation values
runit = xds.rain_rate.units         ### Unit of rain_rate mm/h
rkm = xds.rain_rate.height_km       ### Unit of matrix dimensions km
lon = xds.lon0.data                 ### Coordinate Longitude Matrix
lat = xds.lat0.data                 ### Corrdinate Latitude Matrix
x0 = xds.x0                         ### Matrix of points
y0 = xds.y0

print('Unit of Rain Rate: ',runit)

### 1.2 Read data function

In [ ]:
import numpy as np

def readData(date):
    
    path = 'data/radar/'
    dataset = []
    interval = len(os.listdir(path+str(date)))    
    
    # Original grid dimensions
    nx = 241
    ny = 241

    # Define container
    frames = np.zeros( (interval, nx, ny ) )    
    
    for i in range(interval):
        d = str(path)+str(date)+'/'
        file = (sorted(os.listdir(path+str(date)))[i])
        xds = xr.open_dataset(d+file)
        rr = xds.rain_rate
        frames[i] =  rr
            
    return frames

#### The values loaded for frames correspond to the number of daily observations, and their respective matrices with rainfall rate values.

In [ ]:
frames = readData(day)
print('The frames from dataset:data (times,x,y) -> ',frames.shape)

#### Plot simple example for visualization

In [ ]:
import matplotlib.pyplot as plt
## Plot simple figure from dataset

day = 20150101

figtime = 96
filename = sorted(os.listdir(path+str(day)))[figtime]

plt.figure(figsize=(8,4))
plt.title(filename)
plt.scatter(lon,lat,frames[figtime])
plt.show();

### 1.3 Pre-Processing Function

This function is used to store matrix "x" and "y" coordinates, that have values diffent than "NaN" and greater than the fixed Threshold for each cloud, the values correspond to the characteristic threshold intensity of each point.

<img src="img/dbz_scale.png" align="center" alt="Drawing" style="width: 800px;">

In [ ]:
import pandas as pd

def pre_processing(time1):
    np.warnings.filterwarnings('ignore')
    
    ## thereshold value to track
    threshold = 21.8
    
    rs =  (np.where(time1 != np.nan) and np.where(time1 > threshold))
    rs = np.asanyarray(rs)
    pe = pd.DataFrame({'x1':rs[0],'y1':rs[1]})
        
    return pe

In [ ]:
pdata = pre_processing(frames[96])
pdata.head()

Pre_processing function output with values "x" and "y".

### 1.4 Clustering Process with MeanShift

The purpose of the Mean-Shift clustering technique is to infer the average of clusters by their related density function, where an area calculation is made where the density values are higher and at this point a central point called centroid is measured. The process is performed recursively and is only terminated when the inference value is equal to the previous one.

<img src="img/ms_2d_bw_2.gif" align="left" alt="Drawing" style="width: 400px;">

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth

def clust(time1):
    
    te = time1
    
    if len(te) < 20:
        return None

    bandwidth = estimate_bandwidth(te, quantile=0.3, n_samples=None, random_state=0, n_jobs=None)
    
    if bandwidth > 0:
        ms = MeanShift(bandwidth=5, bin_seeding=None, cluster_all=True, min_bin_freq=1,
    n_jobs=None, seeds=None)
    else:
        ms = MeanShift(bandwidth=10, bin_seeding=None, cluster_all=True, min_bin_freq=1,
    n_jobs=None, seeds=None)

    ms.fit(te)
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    n_clusters_ = len(np.unique(labels))
    te['cluster']=labels
        
#     colors = 10*['r.','g.','b.','c.','k.','y.','m.']
#     for i in range(len(te)):
#         plt.plot(te['x1'][i], te['y1'][i], colors[labels[i]], markersize = 10)
#         plt.title('Estimated number of clusters: %d' % n_clusters_)
        
#     for i, txt in enumerate(range(n_clusters_)):
#         plt.annotate(txt,(cluster_centers[i,0],cluster_centers[i,1]),textcoords="offset points",xytext=(0,10),ha='center')
#     te['cluster']=labels
    
#     plt.scatter(cluster_centers[:,0], cluster_centers[:,1],
#                marker = 'x', s=150, linewidths=10, zorder=10)
    
    return te

In [ ]:
day = 20150101
frames = readData(day)
pdata = pre_processing(frames[90])
clusters = clust(pdata)
clusters.head()

### 1.5 Direction Relations

To define direction relationships between the central points of each cloud and the radar, we use the directional relationship between the points. This approach has as its principle to relate the coordinates of the points and to verify the location between a reference point and a point to be measured. According (Papadias & Sellis, 1994), there are nine possible ways to place a primary point relative to a reference point in the plane, and thus nine disjoint relationships between coordinate pairs that provide complete object coverage.

<img src="img/direction_relations.png" align="center" alt="Drawing" style="width: 800px;">

In [ ]:
def tRelation(p,q):

    north_west       =  p[1] <  q[1] and  p[0] >  q[0] 
    restricted_north =  p[1] == q[1] and  p[0] >  q[0]
    north_east       =  p[1] >  q[1] and  p[0] >  q[0]
    restricted_west  =  p[1] <  q[1] and  p[0] == q[0]
    same_position    =  p[1] == q[1] and  p[0] == q[0]
    restricted_east  =  p[1] >  q[1] and  p[0] == q[0]
    south_west       =  p[1] <  q[1] and  p[0] <  q[0]
    restricted_south =  p[1] == q[1] and  p[0] <  q[0] 
    south_est        =  p[1] >  q[1] and  p[0] <  q[0]

    if north_west == True:
        return ('NW')
    if restricted_north == True:
        return ('RN')
    if north_east == True:
        return ('NE')
    if restricted_west == True:
        return ('RW')
    if same_position == True:
        return('SP')
    if restricted_east == True:
        return ('RE')
    if south_west == True:
        return ('SW')
    if restricted_south == True:
        return ('RS')
    if south_est == True:
        return ('SE')

In [ ]:
x1,y1 = -1.9451030492782593,-61.034568786621094      # p(POINT)
r1,r2 = -3.148556, -59.992000                        # q(RADAR)
p = (x1,y1)                                    #PONTO
q = (r1,r2)                                    #RADAR

print('Point p is ',tRelation(p,q),' of the radar.')

### 1.6 Create Data Process

In [ ]:
from geopy.distance import geodesic

def createData(day,time,clusters,frames):
    
    ##Static Radar Coordinates Value for Topological relation
    radar = (-3.148556, -59.992000)

    if isinstance(clusters,pd.DataFrame):

        FAM1 = pd.DataFrame(columns=['DATETIME',
                                     'N_Cluster','ID_CLUS','LAT','LON','DIST','IND_X','IND_Y',
                                     'T_RELATION','RAIN_FALL','DBz'])
        
        rfall = []
        rlation = []
        ddist = []

        LAT_ = (lat[clusters['x1'],clusters['y1']])
        LON_ = (lon[clusters['x1'],clusters['y1']])
        
        # Calculate distance
        for i, row in clusters.iterrows():
            pt =(str(LAT_[i])+','+str(LON_[i]))
            d = (geodesic(radar, pt).kilometers)
            ddist.append(d)

        # Calculate relation
        for i,row in clusters.iterrows():
            rfall.append(frames[row['x1']][row['y1']])
            r = tRelation((LAT_[i],LON_[i]),radar)
            rlation.append(r)
            
        xds = xr.open_dataset(str(path)+str(day)+'/'+str(sorted(os.listdir(path+str(day)))[time]))
        dtime = xds.start_time.values
        
        FAM1['IND_X'], FAM1['IND_Y'] = clusters['x1'].astype('int16'),clusters['y1'].astype('int16')
        FAM1['LAT'],FAM1['LON'] = LAT_,LON_
        FAM1['DIST'] = ddist
        FAM1['N_Cluster'] = len(clusters['cluster'].unique())
        FAM1['ID_CLUS'] = clusters['cluster']
        FAM1['RAIN_FALL'] = rfall
        FAM1['T_RELATION'] = rlation
        FAM1['DBz'] =  10 * np.log10(200*FAM1['RAIN_FALL']**1.6)
        FAM1['DATETIME'] = pd.to_datetime(dtime)
        FAM1 = FAM1.set_index('DATETIME')
        FAM1 = FAM1.sort_values('ID_CLUS')
        
    else:
        return None

    return FAM1

In [ ]:
day = 20150101
time = 96
frames = readData(day)
pdata = pre_processing(frames[time])
clusters = clust(pdata)
cdata = createData(day,time,clusters,frames[time])
cdata.head()

### 1.7 Extract Centroid by Max RAIN_FALL value

In [ ]:
def centroidData(clus):
    if isinstance(clus,pd.DataFrame):
        centroid = pd.DataFrame()
            
        for i in range(clus['N_Cluster'].max()):
            ct = clus.loc[clus['ID_CLUS'] == i ]
            ct = ct.loc[ct['RAIN_FALL'] == ct['RAIN_FALL'].max()]
            centroid = centroid.append(ct)
    else:
        return None
    return centroid

In [ ]:
ct = centroidData(cdata)
ct.head()

### 1.8 Pre-processing Data

In [ ]:
def run(day):
    data = readData(day)
    cent = pd.DataFrame()
    dados = pd.DataFrame()
    
    for i in range(len(data)):
        proc = pre_processing(data[i]) 
        cluster = clust(proc)
             
        if isinstance(cluster,pd.DataFrame):
            clt = createData(day,i,cluster,data[i])
            dados = dados.append(clt)
            clt2 = centroidData(clt)
            cent = cent.append(clt2)
        else:
            pass
    dados.to_csv('output/clusters/fam_'+str(sorted(os.listdir(path+str(day)))[i][:-3])+'.csv')
    cent.to_csv('output/centroids/centroid_'+str(sorted(os.listdir(path+str(day)))[i][:-3])+'.csv')
    return cent

###  1.9 Run process per Day

In [ ]:
day = 20140928
runner = run(day)

In [ ]:
path = 'data/radar/'

for day in sorted(os.listdir(path)):
    rodada = run(day)
    rodada = None

### 1.9.1 Plot Data

In [ ]:
day = 20150101
plot(96,day)

In [ ]:
import matplotlib.colors as mcolors
from mpl_toolkits.basemap import Basemap
from matplotlib import animation
from matplotlib import markers

def plot(frame,day):

    data = readData(day)
    proc = pre_processing(data[frame])
    clt = clust(proc)

    cdata = createData(day,frame,clt,data[frame])
#     centers = centroid_(day,frame,clt,data[frame])
    centers = centroidData(cdata)

    if centers is not None:
        c_lat = centers['LAT']
        c_lon = centers['LON']
        
        centroid = []
        for i in range(centers['N_Cluster'].max()):
            ct = centers.loc[centers['ID_CLUS'] == i ]
#             ct = ct.loc[ct['RAIN_FALL'] == ct['RAIN_FALL'].max()]
            centroid.append(ct) 
    else:
        centroid = ''
        
    file = path+str(day)+'/'+str(sorted(os.listdir(path+str(day)))[frame])
    xds = xr.open_dataset(file)
    date_time = xds.start_time.data
    date_time = pd.to_datetime(date_time)

    my_coords = [-3.148556, -59.992000]     ## RADAR T1 SIPAM COORDS
    zoom_scale = 2.2                        ## ZOOM SCALE

    bbox = [my_coords[0]-zoom_scale,my_coords[0]+zoom_scale,\
             my_coords[1]-zoom_scale,my_coords[1]+zoom_scale]


    fig, axes = plt.subplots(nrows=1,ncols=1,figsize=(10,10),dpi=100)
    label = 'Rain rate in ' + runit+ ''
    title = 'SIPAM Manaus S-Band Radar :' + str(date_time)

    # draw filled contours.
    clevs = [0, 1, 2.5, 5, 7.5, 10, 15, 20, 30, 40,
                 50, 70, 100]

    cmap_data = [(1.0, 1.0, 1.0),
                     (0.3137255012989044, 0.8156862854957581, 0.8156862854957581),
                     (0.0, 1.0, 1.0),
                     (0.0, 0.8784313797950745, 0.501960813999176),
                     (0.0, 0.7529411911964417, 0.0),
                     (0.501960813999176, 0.8784313797950745, 0.0),
                     (1.0, 1.0, 0.0),
                     (1.0, 0.6274510025978088, 0.0),
                     (1.0, 0.0, 0.0),
                     (1.0, 0.125490203499794, 0.501960813999176),
                     (0.9411764740943909, 0.250980406999588, 1.0),
                     (0.501960813999176, 0.125490203499794, 1.0),
                     (0.250980406999588, 0.250980406999588, 1.0)]

    cmap = mcolors.ListedColormap(cmap_data, 'precipitation')
    norm = mcolors.BoundaryNorm(clevs, cmap.N)
    ax = axes

    m = Basemap(projection='merc',llcrnrlat=bbox[0],urcrnrlat=bbox[1],\
                    llcrnrlon=bbox[2],urcrnrlon=bbox[3],lat_ts=10,resolution='i')

    ## PRECIPTACAO
    xi, yi = m(lon, lat)
    ## SIPAM RADAR
    xm, ym = m(my_coords[1],my_coords[0])
    radar = m.plot(xm,ym, marker='^',color='r', label='RADAR')

    for cent in range(len(centroid)):
        clat, clon, mm_f = centroid[cent]['LAT'].item(),centroid[cent]['LON'].item(),centroid[cent]['RAIN_FALL'].item()
        #clat, clon, mm_f = centroid[1]['LAT'].item(),centroid[1]['LON'].item(),centroid[1]['RAIN_FALL'].item()
        t3x,t3y = m(clon, clat)
        m.plot(t3x,t3y, marker=markers.CARETDOWN, markersize=10, color='k')
#         plt.annotate(str(mm_f)[0:5]+'mm/h', xy=(t3x,t3y),xytext=(t3x+12,t3y+12),rotation=45, size=10)

    m.plot(xm,ym, label='Nº Clusters: ' +str(len(centroid)),marker=markers.CARETDOWN, color='k')

    cs = m.pcolormesh(xi,yi,data[frame], cmap = cmap, norm = norm, ax=ax)

    # # # # Add Grid Lines
    m.drawparallels(np.arange(bbox[0],bbox[1],(bbox[1]-bbox[0])/5),labels=[1,0,0,0],rotation=45, size=(7))
    m.drawmeridians(np.arange(bbox[2],bbox[3],(bbox[3]-bbox[2])/5),labels=[0,0,0,1],rotation=45, size=(7))
    m.drawmapboundary(fill_color='gray')

    m.readshapefile('./img/hidro/lineaire_1km', 'hidro10km', color='b', linewidth=1)
    # # # # Add Colorbar
    cbar = m.colorbar(cs, location='bottom', pad="10%")
    cbar.set_label(label)

    # # # # # Add Title
    plt.title(title)
    plt.legend()
    plt.ylabel('Longitude', labelpad=40)
    plt.xlabel('Latitude', labelpad=60)

    plt.savefig('radar_image/'+ sorted(os.listdir(path+str(day)))[frame]+'.png')

    plt.show()

In [ ]:
## CREATE ALL FIGS
for i in range(len(frames)):
    plot(i,day)

### 3. Tracking

In [ ]:
import glob
from scipy.spatial.distance import cdist

In [ ]:
l = [pd.read_csv(filename) for filename in sorted(glob.glob("output/centroids/*.csv"))]
dados = pd.concat(l, axis=0, sort=False)
dados['DATETIME'] =  pd.to_datetime(dados['DATETIME'], format='%Y-%m-%d %H:%M:%S')
dados = dados.set_index('DATETIME')
dados.index = dados.index.map(lambda x: x.replace(second=0))

freqtime = pd.date_range(start=dados.index.min(), end=dados.index.max(), freq='12T')

time = []

for i in range(len(freqtime)):
    t = dados.loc[dados.index == freqtime[i]]
    time.append(t)

In [ ]:
def calcDist(t1,t2):
    
    if len(t1) == 0 or len(t2) == 0:
        return None
    
    time1 = t1
    time2 = t2

    t1 = t1[['ID_CLUS','IND_X','IND_Y']]
    t1_data = t1.values.tolist()

    t2 = t2[['ID_CLUS','IND_X','IND_Y']]
    t2_data = t2.values.tolist()

    t1_ = pd.DataFrame(data=t1_data, columns=['T1', 'X', 'Y'])
    t2_ = pd.DataFrame(data=t2_data, columns=['T2', 'X', 'Y'])

    mask = cdist(t2_[['X', 'Y']].values, t1_[['X', 'Y']].values,'euclidean') < 5

    arr = []
    def zone(x):
        arr.append ( t1_[x].T1.values[0] if x.any() else np.nan)
        
    result = pd.DataFrame()
    
    np.apply_along_axis(zone, True, mask)
    result['T1'] = arr
    result['T2'] = t2_.T2.values

    
    result['START_TIME'] = t1.index[0]
    result['END_TIME'] = t2.index[0]
    result['X2'] = (t2.IND_X.values)
    result['Y2'] = (t2.IND_Y.values)
    result['LAT2'] = (time2.LAT.values)
    result['LON2'] = (time2.LON.values)

    x1,y1 = [],[]
    lat1,lon1 = [],[]
    lat2,lon2 = [],[]
    dist = []

    for i in range(len(arr)):
        a = time1.loc[time1.ID_CLUS == arr[i]]
        b = time2.loc[time2.ID_CLUS == arr[i]]
        x1.append(a['IND_X'].values)
        y1.append(a['IND_Y'].values)

        lat1.append(a['LAT'].values)
        lon1.append(a['LON'].values)

    result['X1']= x1
    result['Y1']= y1
    result['LAT1'] = lat1
    result['LON1'] = lon1

    result['X1'] = result['X1'].str[0]
    result['Y1'] = result['Y1'].str[0]
    result['LAT1'] = result['LAT1'].str[0]
    result['LON1'] = result['LON1'].str[0]
    
    result = result.sort_values('T1')
    result = result.dropna()
    result['T1'] = result['T1'].astype(int)
    result['X1'] = result['X1'].astype(int)
    result['Y1'] = result['Y1'].astype(int)
    
    dist,vel,rel = [],[],[]
    
    ## CALC DIST
    for i,row in result.iterrows():
        p1 = (str(row.LAT1)+','+str(row.LON1))
        p2 = (str(row.LAT2)+','+str(row.LON2))
        dist.append(geodesic(p1, p2).kilometers)
        d1 = (row.LAT1,row.LON1)
        d2 = (row.LAT2,row.LON2)
        rel.append(tRelation(d1,d2))
    
    result['DIST'] = dist
    result['REL'] = rel

    ## CALV VEL
    for i,row in result.iterrows():
        vel.append(row.DIST/0.2)
    result['VELM'] = vel
    result = result[['START_TIME','END_TIME','T1','T2','X1','Y1','X2','Y2','LAT1','LON1','LAT2','LON2','DIST','VELM','REL']]

    return result

In [ ]:
track = pd.DataFrame()

aux = []

for i in range(len(time)):
    if len(time[i]) > 0:
        aux.append(i)
for i in range(len(aux)-1):
    track = track.append(calcDist(time[aux[i]],time[aux[i+1]]))

In [ ]:
track.head()
track.to_csv('output/tracking/tracking.csv')

### 3. Statistics and Results

In [ ]:
import glob
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

In [ ]:
l = [pd.read_csv(filename) for filename in sorted(glob.glob("output/centroids/*.csv"))]
dados = pd.concat(l, axis=0, sort=False)
dados = dados.loc[:, ~dados.columns.str.contains('^Unnamed')]
dados['DATETIME'] =  pd.to_datetime(dados['DATETIME'], format='%Y-%m-%d %H:%M:%S')
dados = dados.set_index('DATETIME')

In [ ]:
dados.head()

In [ ]:
tidy_data2015 = pd.DataFrame()
tidy_data2015 = dados

# data2014 = dados.loc[dados['YEAR'] == 2014]
# data2015 = dados.loc[dados['YEAR'] == 2015]

# tidy_data2014 = pd.DataFrame()
# tidy_data2015 = pd.DataFrame()
# tidy_data2015 = dados

In [ ]:
tidy_data2015.head()

In [ ]:
## FOR 2014
data2014 = data2014[['YEAR','MONTH','DAY','HOUR','MINUTE','N_Cluster','ID_CLUS','T_RELATION','RAIN_FALL','DBz']]

data2014['YEAR'] = data2014['YEAR'].astype(int)
data2014['MONTH'] = data2014['MONTH'].astype(int)
data2014['DAY'] = data2014['DAY'].astype(int)
data2014['HOUR'] = data2014['HOUR'].astype(int)
data2014['MINUTE'] = data2014['MINUTE'].astype(int)
data2014['N_Cluster'] = data2014['N_Cluster'].astype(int)
data2014['ID_CLUS'] = data2014['ID_CLUS'].astype(int)
data2014['T_RELATION'] = data2014['T_RELATION'].astype(str)

data2014['DateTime'] = data2014.apply(lambda row: datetime(
                              row['YEAR'], row['MONTH'], row['DAY'], row['HOUR'], row['MINUTE']), axis=1)

data2014 = data2014.drop(columns=['YEAR','MONTH','DAY','HOUR','MINUTE'])
data2014 = data2014.set_index('DateTime')

tidy_data2014 = data2014

## FOR 2015
data2015 = data2015[['YEAR','MONTH','DAY','HOUR','MINUTE','N_Cluster','ID_CLUS','T_RELATION','RAIN_FALL','DBz']]

data2015['YEAR'] = data2015['YEAR'].astype(int)
data2015['MONTH'] = data2015['MONTH'].astype(int)
data2015['DAY'] = data2015['DAY'].astype(int)
data2015['HOUR'] = data2015['HOUR'].astype(int)
data2015['MINUTE'] = data2015['MINUTE'].astype(int)
data2015['N_Cluster'] = data2015['N_Cluster'].astype(int)
data2015['ID_CLUS'] = data2015['ID_CLUS'].astype(int)
data2015['T_RELATION'] = data2015['T_RELATION'].astype(str)

data2015['DateTime'] = data2015.apply(lambda row: datetime(
                              row['YEAR'], row['MONTH'], row['DAY'], row['HOUR'], row['MINUTE']), axis=1)

data2015 = data2015.drop(columns=['YEAR','MONTH','DAY','HOUR','MINUTE'])
data2015 = data2015.set_index('DateTime')

tidy_data2015 = data2015

In [ ]:
tidy_data2014.head()

In [ ]:
tidy_data2015.head()

In [ ]:
# NCluster = tidy_data2014.resample('min').mean()
# NCluster = NCluster.dropna()
# NCluster = NCluster.resample('M')[['N_Cluster']].sum()

NCluster5 = tidy_data2015.resample('min').mean()
NCluster5 = NCluster5.dropna()
NCluster5 = NCluster5.resample('M')[['N_Cluster']].sum()

# cdf2014 = pd.DataFrame({'date':NCluster.index, 'clusters':NCluster.values.ravel()})
cdf2015 = pd.DataFrame({'date':NCluster5.index, 'clusters':NCluster5.values.ravel()})

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cdf2014.date, y=cdf2014['clusters'], name="2014",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=cdf2014.date, y=cdf2015['clusters'], name="2015",
                         line_color='dimgray'))


fig.update_layout(
    title=go.layout.Title(
        text="Ocorrências de Clusters para os anos de 2014 e 2015",
        xref="paper",
        xanchor = "auto",
        x=0,
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Número de clusters",
            font=dict(
                family="Courier New, monospace",
                size=14,
                color="#7f7f7f"
            )
        )
    )
)

In [ ]:
NRain = tidy_data2014.resample('min').mean()
NRain = NRain.dropna()
NRain = NRain.resample('M')[['RAIN_FALL']].mean()

NRain5 = tidy_data2015.resample('min').mean()
NRain5 = NRain5.dropna()
NRain5 = NRain5.resample('M')[['RAIN_FALL']].mean()

rdf2014 = pd.DataFrame({'date':NRain.index, 'clusters':NRain.values.ravel()})
rdf2015 = pd.DataFrame({'date':NRain5.index, 'clusters':NRain5.values.ravel()})

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=rdf2014.date, y=rdf2014['clusters'], name="2014",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=rdf2014.date, y=rdf2015['clusters'], name="2015",
                         line_color='dimgray'))


fig.update_layout(
    title=go.layout.Title(
        text="Precipitação média mensal de 2014 e 2015",
        xref="paper",
        xanchor = "auto",
        x=0,
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Precipitação média por clusters",
            font=dict(
                family="Courier New, monospace",
                size=14,
                color="#7f7f7f"
            )
        )
    )
)

In [ ]:
an2014 = tidy_data2014.resample('Y')['T_RELATION'].value_counts()

df2014 = pd.DataFrame({'Relação':an2014.index.get_level_values('T_RELATION').values, 'Valores':an2014.values.ravel()})

fig = px.bar(df2014, x='Relação', y='Valores', color='Relação', labels='Relação')
fig.update_layout(title_text='Relações direcionais para o ano de 2014')
fig.show()

In [ ]:
an2015 = tidy_data2015.resample('Y')['T_RELATION'].value_counts()

an2015 = pd.DataFrame({'Relação':an2015.index.get_level_values('T_RELATION').values, 'Valores':an2015.values.ravel()})

fig = px.bar(an2015, x='Relação', y='Valores', color='Relação', labels='Relação')
fig.update_layout(title_text='Relações direcionais para o ano de 2015')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df2014['Relação'],
    y=df2014['Valores'],
    name='2014',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=an2015['Relação'],
    y=an2015['Valores'],
    name='2015',
    marker_color='lightsalmon'
))

fig.update_layout(title_text='Relações direcionais para o ano de 2015', barmode='group')
fig.show()

In [ ]:
men2014 = tidy_data2014.resample('M')['T_RELATION'].value_counts()

m2014 = pd.DataFrame({'Data': men2014.index.get_level_values('DateTime').values.ravel(),
                      'Relação':men2014.index.get_level_values('T_RELATION').values.ravel(),
                      'Valores':men2014.values.ravel()})

m2014['Data'] = pd.to_datetime(m2014['Data'])

fig = px.bar(m2014, x='Data', y='Valores', color='Relação', labels='Relação',barmode='group')
fig.update_layout(title_text='Relações direcionais para o ano de 2014')
fig.show()

In [ ]:
men2015 = tidy_data2015.resample('M')['T_RELATION'].value_counts()

m2015 = pd.DataFrame({'Data': men2015.index.get_level_values('DateTime').values.ravel(),
                      'Relação':men2015.index.get_level_values('T_RELATION').values.ravel(),
                      'Valores':men2015.values.ravel()})

m2015['Data'] = pd.to_datetime(m2015['Data'])

fig = px.bar(m2015, x='Data', y='Valores', color='Relação', labels='Relação',barmode='group')
fig.update_layout(title_text='Relações direcionais para o ano de 2015')
fig.show()

### FORTRACC

In [ ]:
import glob
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go

In [ ]:
f = [pd.read_csv(filename, delimiter=r"\s+") for filename in sorted(glob.glob("output/fortracc/*.txt"))]
fortracc = pd.concat(f, axis=0, sort=False)

In [ ]:
fortracc.head()

In [ ]:
## FOR 2015
fdata = pd.DataFrame(columns=['YEAR','MONTH','DAY','HOUR','MINUTE','N_Cluster','REFLECT','PRECIPIT'])

fdata['YEAR'] = fortracc['YEAR'].astype('int16')
fdata['MONTH'] = fortracc['MONTH'].astype('int16')
fdata['DAY'] = fortracc['DAY'].astype('int16')
fdata['HOUR'] = fortracc['HOUR'].astype('int16')
fdata['MINUTE'] = fortracc['MINUTE'].astype('int16')
fdata['N_Cluster'] = fortracc['N_Cluster'].astype('int16')
fdata['REFLECT'] = fortracc['REFLECT'].astype('float32')
fdata['PRECIPIT'] = fortracc['PRECIPIT'].astype('float32')

fdata['DATETIME'] = fdata.apply(lambda row: datetime(
                              row['YEAR'], row['MONTH'], row['DAY'], row['HOUR'], row['MINUTE']), axis=1)

fdata = fdata.drop(columns=['YEAR','MONTH','DAY','HOUR','MINUTE'])
fdata = fdata.set_index('DATETIME')

tidy_fortracc = fdata.sort_values(by='DATETIME')

In [ ]:
tidy_fortracc.head()

In [ ]:
FCluster = tidy_fortracc.resample('min').mean()
FCluster = FCluster.dropna()
FCluster = FCluster.resample('M')[['N_Cluster']].sum()

FCluster2014 = pd.DataFrame({'date':FCluster.index, 'clusters':FCluster.values.ravel()})

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=FCluster2014.date, y=FCluster2014['clusters'], name="ForTraCC",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=cdf2014.date, y=cdf2014['clusters'], name="MS 2014",
                         line_color='dimgray'))

fig.add_trace(go.Scatter(x=cdf2014.date, y=cdf2015['clusters'], name="MS 2015",
                         line_color='blueviolet'))


fig.update_layout(
    title=go.layout.Title(
        text="Média mensal do número de Clusters",
        xref="paper",
        xanchor = "auto",
        x=0,
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Número de Clusters",
            font=dict(
                family="Courier New, monospace",
                size=14,
                color="#7f7f7f"
            )
        )
    )
)

#### CONCLUSIONS

#### REFERENCES

* CHENG, Yizong. Mean shift, mode seeking, and clustering. IEEE transactions on pattern analysis and machine intelligence, v. 17, n. 8, p. 790-799, 1995.

* FRANK, Andrew U. Qualitative spatial reasoning: Cardinal directions as an example. International Journal of Geographical Information Science, v. 10, n. 3, p. 269-290, 1996.

* Mean Shift Clustering Overview by Matt Nedrich https://spin.atomicobject.com/2015/05/26/mean-shift-clustering/

* The NWS NEXRAD WSR88-D Radar: https://web.archive.org/web/20160113151652/http://www.desktopdoppler.com/help/nws-nexrad.htm#rainfall%20rates

* PAPADIAS, Dimitris; THEODORIDIS, Yannis. Spatial relations, minimum bounding rectangles, and spatial data structures. International Journal of Geographical Information Science, v. 11, n. 2, p. 111-138, 1997.